In [36]:
import librosa
import numpy as np
from CQT import cqt

In [58]:
y, sr = librosa.load('data/MusicDelta_Hendrix_STEM_04.wav')
C = np.abs(librosa.cqt(y, sr=16000, fmin=librosa.note_to_hz('C1')
                       ,n_bins=24)) #nbins upto 113

# C = np.abs(cqt(y, fmin=librosa.note_to_hz('A1'), fmax=librosa.note_to_hz('A8'), bins=128, fs=sr)['spCQT'])

In [59]:
y.shape
C.shape

(24, 855)